<a href="https://colab.research.google.com/github/myrondza/Data-Science-Machine-Learning-Deep-Learning-AI-Guide-Algorithms/blob/master/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatBot (Keras & Tensorflow)

In [0]:
import pickle
import numpy as np

In [197]:
a = [
    (['Myron','went','to','work'],['Is','Myron','at','work','?'],'yes'),
    (['Myron','did','not','go','to','office'],['Is','Myron','at','office','?'],'no'),
    (['John','did','not','go','to','the','party'],['Is','John','at','the','party','?'],'no')
    ]

file_Name = "train_basic.txt"
fileObject = open(file_Name,'wb') 
pickle.dump(a,fileObject)  

fileObject.close()
fileObject = open(file_Name,'rb')  
train_data = pickle.load(fileObject)  
train_data

[(['Myron', 'went', 'to', 'work'], ['Is', 'Myron', 'at', 'work', '?'], 'yes'),
 (['Myron', 'did', 'not', 'go', 'to', 'office'],
  ['Is', 'Myron', 'at', 'office', '?'],
  'no'),
 (['John', 'did', 'not', 'go', 'to', 'the', 'party'],
  ['Is', 'John', 'at', 'the', 'party', '?'],
  'no')]

In [198]:
len(train_data)

3

In [199]:
b = [
    (['Myron','did','not','go','to','office'],['Is','Myron','at','office','?'],'no')
    ]

file_Name = "test_basic.txt"
fileObject = open(file_Name,'wb') 
pickle.dump(b,fileObject)  

fileObject.close()
fileObject = open(file_Name,'rb')  
test_data = pickle.load(fileObject)  
test_data

[(['Myron', 'did', 'not', 'go', 'to', 'office'],
  ['Is', 'Myron', 'at', 'office', '?'],
  'no')]

In [200]:
len(test_data)

1

In [201]:
train_data[0]

(['Myron', 'went', 'to', 'work'], ['Is', 'Myron', 'at', 'work', '?'], 'yes')

In [202]:
test_data[0]

(['Myron', 'did', 'not', 'go', 'to', 'office'],
 ['Is', 'Myron', 'at', 'office', '?'],
 'no')

In [0]:
vocab = set()
for story, question, answer in train_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [0]:
vocab.add('no')
vocab.add('yes')

In [205]:
vocab

{'?',
 'Is',
 'John',
 'Myron',
 'at',
 'did',
 'go',
 'no',
 'not',
 'office',
 'party',
 'the',
 'to',
 'went',
 'work',
 'yes'}

In [0]:
vocab_len = len(vocab) + 1

In [0]:
all_data = test_data + train_data
all_story_lens = [len(data[0]) for data in all_data]
max_story_len = (max(all_story_lens))
max_question_len = max([len(data[1]) for data in all_data])

In [0]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [0]:
tokenizer = Tokenizer(filters = [])
tokenizer.fit_on_texts(vocab)

In [210]:
tokenizer.word_index

{'?': 1,
 'at': 8,
 'did': 2,
 'go': 15,
 'is': 12,
 'john': 9,
 'myron': 6,
 'no': 11,
 'not': 3,
 'office': 5,
 'party': 14,
 'the': 10,
 'to': 7,
 'went': 4,
 'work': 16,
 'yes': 13}

In [0]:
train_story_text = []
train_question_text = []
train_answers = []

In [0]:
for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question) 
    train_answers.append(answer)

In [0]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [0]:
def vectorize_stories(data,word_index = tokenizer.word_index, max_story_len = max_story_len, max_question_len = max_question_len):
    #vectorized stories:
    X = []
    #vectorized questions:
    Xq = []
    #vectorized answers:
    Y = []
    
    for story, question, answer in data:
        #Getting indexes for each word in the story
        x = [word_index[word.lower()] for word in story]
        #Getting indexes for each word in the story
        xq = [word_index[word.lower()] for word in question]
        #For the answers
        y = np.zeros(len(word_index) + 1) #Index 0 Reserved when padding the sequences
        y[word_index[answer]] = 1
        
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
    #Now we have to pad these sequences:
    return(pad_sequences(X,maxlen=max_story_len), pad_sequences(Xq, maxlen=max_question_len), np.array(Y))
       

In [0]:
inputs_train, questions_train, answers_train = vectorize_stories(train_data)

In [0]:
inputs_test, questions_test, answers_test = vectorize_stories(test_data)

In [217]:
inputs_train[0]

array([ 0,  0,  0,  6,  4,  7, 16], dtype=int32)

In [218]:
train_story_text[0]

['Myron', 'went', 'to', 'work']

In [219]:
train_story_seq[0]

[6, 4, 7, 16]

In [0]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate, LSTM

In [0]:
input_sequence = Input((max_story_len,))
question = Input((max_question_len,))

In [0]:
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_len,output_dim = 64)) #From paper
input_encoder_m.add(Dropout(0.3))

input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_len,output_dim = max_question_len)) #From paper
input_encoder_c.add(Dropout(0.3))

question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_len,output_dim = 64,input_length=max_question_len)) #From paper
question_encoder.add(Dropout(0.3))

input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

match = dot([input_encoded_m,question_encoded], axes = (2,2))
match = Activation('softmax')(match)

response = add([match,input_encoded_c])
response = Permute((2,1))(response)

answer = concatenate([response, question_encoded])

In [223]:
answer

<tf.Tensor 'concatenate_6/concat:0' shape=(?, 6, 71) dtype=float32>

In [224]:
answer = LSTM(32)(answer)
answer = Dropout(0.5)(answer)
answer = Dense(vocab_len)(answer)
answer = Activation('softmax')(answer)
model = Model([input_sequence,question], answer)
model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 7)            0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, 6)            0                                            
__________________________________________________________________________________________________
sequential_16 (Sequential)      multiple             1088        input_13[0][0]                   
__________________________________________________________________________________________________
sequential_18 (Sequential)      (None, 6, 64)        1088        input_14[0][0]                   
__________________________________________________________________________________________________
dot_6 (Dot

In [225]:
history = model.fit([inputs_train,questions_train],answers_train, batch_size = 32, epochs = 1000, validation_data = ([inputs_test,questions_test],answers_test))

Train on 3 samples, validate on 1 samples
Epoch 1/1000
3/3 [==============================] - 2s 592ms/step - loss: 2.8772 - acc: 0.0000e+00 - val_loss: 2.8104 - val_acc: 0.0000e+00
Epoch 2/1000
3/3 [==============================] - 0s 4ms/step - loss: 2.8023 - acc: 0.3333 - val_loss: 2.7630 - val_acc: 1.0000
Epoch 3/1000
3/3 [==============================] - 0s 3ms/step - loss: 2.7847 - acc: 0.0000e+00 - val_loss: 2.7273 - val_acc: 1.0000
Epoch 4/1000
3/3 [==============================] - 0s 4ms/step - loss: 2.7761 - acc: 0.3333 - val_loss: 2.6897 - val_acc: 1.0000
Epoch 5/1000
3/3 [==============================] - 0s 3ms/step - loss: 2.7278 - acc: 0.3333 - val_loss: 2.6539 - val_acc: 1.0000
Epoch 6/1000
3/3 [==============================] - 0s 4ms/step - loss: 2.6948 - acc: 0.6667 - val_loss: 2.6192 - val_acc: 1.0000
Epoch 7/1000
3/3 [==============================] - 0s 3ms/step - loss: 2.6775 - acc: 0.6667 - val_loss: 2.5806 - val_acc: 1.0000
Epoch 8/1000
3/3 [================

In [0]:
filename = 'chatbot_epochs.h5'
model.save(filename)

In [0]:
model.load_weights('chatbot_epochs.h5')

In [0]:
pred_results = model.predict(([inputs_test,questions_test]))

In [229]:
test_data[0]

(['Myron', 'did', 'not', 'go', 'to', 'office'],
 ['Is', 'Myron', 'at', 'office', '?'],
 'no')

In [230]:
pred_results[0]

array([2.0986235e-08, 1.8033216e-08, 8.9533492e-09, 1.1278227e-08,
       2.4097119e-08, 1.8310141e-08, 1.7575205e-08, 1.0139728e-08,
       1.8747681e-08, 7.6482651e-09, 6.4320753e-09, 9.9999845e-01,
       1.0234938e-08, 1.3515674e-06, 1.8302844e-08, 2.2311882e-08,
       6.6580581e-09], dtype=float32)

In [231]:
val_max = np.argmax(pred_results[0])
for key,val in tokenizer.word_index.items():
    if val == val_max:
        k = key
print(k)

no


In [232]:
pred_results[0][val_max]

0.99999845

In [242]:
my_story = 'John went to office'
my_story.split()

['John', 'went', 'to', 'office']

In [246]:
my_question = 'Is John at office ?'
my_question.split()

['Is', 'John', 'at', 'office', '?']

In [0]:
my_data = [(my_story.split(), my_question.split(),'yes')]

In [0]:
my_story, my_ques, my_ans = vectorize_stories(my_data)

In [0]:
pred_results = model.predict(([my_story,my_ques]))

In [0]:
val_max = np.argmax(pred_results[0])

In [251]:
for key,val in tokenizer.word_index.items():
    if val == val_max:
        k = key
print(k)

yes


In [252]:
pred_results[0][val_max]

0.9999938